Here i launch runs of segmentation model with new data generator using wandb

In [1]:
!pip install wandb -Uq

     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 65.4 MB/s 
     |████████████████████████████████| 166 kB 67.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 70.2 MB/s 
     |████████████████████████████████| 162 kB 65.9 MB/s 
     |████████████████████████████████| 162 kB 53.7 MB/s 
     |████████████████████████████████| 158 kB 70.4 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 68.2 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████████| 157 kB 73.0 MB/s 
     |████████████████████████████████| 157 kB 73.3 MB/s 
     |████████████████████████████████| 157 kB 69.7 MB/s 
     |████████████████████████████████| 157 kB 75.3 MB/s 
     |████████████████████████████████| 156 kB 76.4 MB/s 


In [2]:
!pip install git+https://github.com/KonstantinUshenin/nndt.git@add_data_generator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/KonstantinUshenin/nndt.git (to revision add_data_generator) to /tmp/pip-req-build-n9my7hrl
  Running command git clone -q https://github.com/KonstantinUshenin/nndt.git /tmp/pip-req-build-n9my7hrl
  Running command git checkout -b add_data_generator --track origin/add_data_generator
  Switched to a new branch 'add_data_generator'
  Branch 'add_data_generator' set up to track remote branch 'add_data_generator' from 'origin'.
     |████████████████████████████████| 41 kB 632 kB/s 
     |████████████████████████████████| 350 kB 9.0 MB/s 
     |████████████████████████████████| 145 kB 62.1 MB/s 
     |████████████████████████████████| 79.3 MB 1.4 MB/s 
     |████████████████████████████████| 1.5 MB 25.6 MB/s 
     |████████████████████████████████| 246 kB 61.1 MB/s 
     |████████████████████████████████| 62 kB 695 kB/s 
  Installing build dependencies ... done
  G

In [3]:
import nndt
import nndt.space2 as spc
from nndt.datasets import ACDC
nndt.__version__
from tqdm import tqdm
import jax
import jax.numpy as jnp
from nndt.trainable_task import SurfaceSegmentation
from nndt.vizualize import BasicVizualization
import optax
from nndt.datagen import DataGenForSegmentation


ACDC().load()

Downloading...
From: https://drive.google.com/uc?export=download&id=1wOYib-CBGcx_x-ap0eBu6mstHbICLxKG
To: /content/.datasets/ACDC_5/temp.zip
100%|██████████| 45.6M/45.6M [00:00<00:00, 61.9MB/s]


Loading complete


In [6]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
import wandb
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "/content/drive/MyDrive/Colab Notebooks/1_work_experiments_wandb.ipynb"
wandb.login()

True

# Space initialzation

In [10]:
key = jax.random.PRNGKey(42)

space = spc.load_from_path("./.datasets/ACDC_5")
space = spc.split_node_test_train(key, space, test_size=0.4)# 2 for test
space.preload("shift_and_scale")
print(space.print('source'))

S:space 0.0.2
├── G:test ((-0.76, -0.73, -0.50), (0.76, 0.73, 0.50))
│   ├── O3D:patient049 ((-0.76, -0.73, -0.50), (0.76, 0.73, 0.50))
│   │   ├── FS:colored_obj mesh_obj^ ./.datasets/ACDC_5/patient049/colored.obj
│   │   ├── FS:sdf_npy sdt^ ./.datasets/ACDC_5/patient049/sdf.npy
│   │   └── TR:transform shift_and_scale
│   └── O3D:patient069 ((-0.58, -0.48, -0.48), (0.58, 0.48, 0.48))
│       ├── FS:colored_obj mesh_obj^ ./.datasets/ACDC_5/patient069/colored.obj
│       ├── FS:sdf_npy sdt^ ./.datasets/ACDC_5/patient069/sdf.npy
│       └── TR:transform shift_and_scale
└── G:train ((-0.75, -0.78, -0.83), (0.75, 0.78, 0.83))
    ├── O3D:patient009 ((-0.75, -0.71, -0.77), (0.75, 0.71, 0.77))
    │   ├── FS:colored_obj mesh_obj^ ./.datasets/ACDC_5/patient009/colored.obj
    │   ├── FS:sdf_npy sdt^ ./.datasets/ACDC_5/patient009/sdf.npy
    │   └── TR:transform shift_and_scale
    ├── O3D:patient029 ((-0.75, -0.78, -0.83), (0.75, 0.78, 0.83))
    │   ├── FS:colored_obj mesh_obj^ ./.datasets/

# Creating supplementary functions for sweep

In [11]:
def build_dataset(config):
    train_gen = DataGenForSegmentation(space.train, augment=True,
                                       cube_scale = config.cube_scale,
                                       shift_sigma = config.shift_sigma,
                                       scale_range = config.scale_range,
                                       shift_mul = config.shift_mul,
                                       rotate_angle= config.rotate_angle,
                                       ) 
    test_gen = DataGenForSegmentation(space.test, augment=False)

    train_X = []
    train_Y = []
    test_X = []
    test_Y = []

    key = jax.random.PRNGKey(42)

    for i in tqdm(range(500)):
        key, subkey = jax.random.split(key)
        train_res = train_gen.get(subkey, i)
        train_X.append(train_res[0])
        train_Y.append(train_res[1])

    for i in tqdm(range(10)):
        key, subkey = jax.random.split(key)
        test_res = test_gen.get(subkey, i*33)
        test_X.append(test_res[0])
        test_Y.append(test_res[1])

    train_X = jnp.stack(train_X, axis=0)
    train_Y = jnp.stack(train_Y, axis=0)
    test_X = jnp.concatenate(test_X, axis=0)
    test_Y = jnp.concatenate(test_Y, axis=0)

    #train_X.shape, train_Y.shape, test_X.shape, test_Y.shape
    return train_X, train_Y, test_X, test_Y

In [12]:
def init_network(config):
    # NN initialization
    task = SurfaceSegmentation(spacing=(16,16,16),
                                conv_kernel=32,
                                conv_depth=4,
                                num_classes=3,
                                batch_size=128)
    rng = jax.random.PRNGKey(42)
    params, F = task.init_and_functions(rng)

    opt = optax.adam(0.001)
    opt_state = opt.init(params)


    viz = BasicVizualization('experiment', 'e1', print_on_each_epoch=20)


    return params, F, opt, opt_state, viz, rng

In [13]:
sweep_config = {
    'method': 'grid'
    }
parameters_dict = {
    'cube_scale': {
        'values': [1, 0.5, 1.5]
        },
    'shift_sigma': {
        'values': [0.1, 0.05, 0.2]
        },
    'scale_range': {
          'values': [0.03, 0.01, 0.05]
        },
    'shift_mul': {
          'values': [0, 1, 2, 3, 5]
        },
    'rotate_angle': {
          'values': [1]
        },
    }

sweep_config['parameters'] = parameters_dict


In [14]:
parameters_dict.update({
    'learning_rate': {
        'value' : 0.001
        },
    'batch_size': {
       'value' : 128
      },
    'conv_depth':{
        'value' :3
      },
    'conv_kernel':{
        'value' :32
    },
    'spacing': {
        'value' :(16,16,16)
    }
    })

In [15]:
import pprint
pprint.pprint(sweep_config)

{'method': 'grid',
 'parameters': {'batch_size': {'value': 128},
                'conv_depth': {'value': 3},
                'conv_kernel': {'value': 32},
                'cube_scale': {'values': [1, 0.5, 1.5]},
                'learning_rate': {'value': 0.001},
                'rotate_angle': {'values': [1]},
                'scale_range': {'values': [0.03, 0.01, 0.05]},
                'shift_mul': {'values': [0, 1, 2, 3, 5]},
                'shift_sigma': {'values': [0.1, 0.05, 0.2]},
                'spacing': {'value': (16, 16, 16)}}}


In [16]:
sweep_id = wandb.sweep(sweep_config, project="First_datagen_runs")

Create sweep with ID: iindu0yq
Sweep URL: https://wandb.ai/dopervigest/First_datagen_runs/sweeps/iindu0yq


In [17]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        train_X, train_Y, test_X, test_Y = build_dataset(config)
        params, F, opt, opt_state, viz, rng = init_network(config)

        @jax.jit
        def train_step(params, rng, opt_state, X, Y):
            loss, grads = jax.value_and_grad(F.main_loss)(params, rng, X, Y)
            updates, opt_state = opt.update(grads, opt_state)
            params = optax.apply_updates(params, updates)
            accuracy = F.metric_accuracy(params, rng, X, Y)

            return loss, accuracy, params, rng, opt_state


        @jax.jit
        def eval_step(params, rng, X, Y):
            loss = F.main_loss(params, rng, X, Y)
            accuracy = F.metric_accuracy(params, rng, X, Y)
            return loss, accuracy


        for epoch in viz.iter(10001):

            ep = epoch % train_X.shape[0]
            loss, accuracy, params, rng, opt_state = train_step(params, rng, opt_state, train_X[ep], train_Y[ep])
            val_loss, val_accuracy = eval_step(params, rng, test_X, test_Y)

            viz.record({"loss": float(loss), "accuracy": float(accuracy),
                        "val_loss": float(val_loss), "val_accuracy": float(val_accuracy)})
            

            wandb.log({"loss": float(loss),
                       "accuracy": float(accuracy),
                       "val_loss": float(val_loss),
                       "val_accuracy": float(val_accuracy),
                       "epoch": epoch})   

            if viz.is_print_on_epoch(epoch):
                viz.draw_loss("TRAIN_LOSS", viz._records["loss"])
                viz.draw_loss("TRAIN_ACCURACY", viz._records["accuracy"])
                viz.draw_loss("TEST_LOSS", viz._records["val_loss"])
                viz.draw_loss("TEST_ACCURACY", viz._records["val_accuracy"])

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 4crmyxli with config:
wandb: 	batch_size: 128
wandb: 	conv_depth: 3
wandb: 	conv_kernel: 32
wandb: 	cube_scale: 1
wandb: 	learning_rate: 0.001
wandb: 	rotate_angle: 1
wandb: 	scale_range: 0.03
wandb: 	shift_mul: 0
wandb: 	shift_sigma: 0.1
wandb: 	spacing: [16, 16, 16]
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /content/drive/MyDrive/Colab Notebooks/1_work_experiments_wandb.ipynb.


100%|██████████| 10/10 [00:01<00:00,  8.30it/s]


[E:0,T:0.0] 

[E:20,T:7.4] loss=1.0085853338241577, accuracy=0.4343434274196625, val_loss=1.037904143333435, val_accuracy=0.4590908885002136, 

[E:40,T:11.0] loss=0.8188595771789551, accuracy=0.6969696879386902, val_loss=0.9905663132667542, val_accuracy=0.5363636016845703, 

[E:60,T:14.6] loss=0.7993839383125305, accuracy=0.7575757503509521, val_loss=0.9248338937759399, val_accuracy=0.603030264377594, 

[E:80,T:18.2] loss=0.8344186544418335, accuracy=0.747474730014801, val_loss=0.8908849954605103, val_accuracy=0.6393939256668091, 

[E:100,T:21.9] loss=0.7825993895530701, accuracy=0.7777777910232544, val_loss=0.8868700265884399, val_accuracy=0.6424242258071899, 

[E:120,T:25.4] loss=0.7852554321289062, accuracy=0.7676767706871033, val_loss=0.9152733087539673, val_accuracy=0.6106060147285461, 

[E:140,T:29.0] loss=0.7942102551460266, accuracy=0.7373737096786499, val_loss=0.9108477830886841, val_accuracy=0.6227272748947144, 

[E:160,T:32.6] loss=0.8280266523361206, accuracy=0.737373709678

accuracy,▁▂▃▅▅▅▅▇▆▆▅▇▆▇▆▇▇▇█▆▇▇▇█▆▇▆▆█▇▆▆▇▆▇▇▇▇▇█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▄▄▄▃▃▃▃▂▃▂▃▂▂▁▁▃▂▃▂▁▃▂▃▂▁▂▃▂▂▃▂▂▃▁▂▁
val_accuracy,▁▁▃▄▄▅▆▆▇▇▅▅▇▇▇▇▇▇▇▇▇█▇███▆█▇▇▇▇████▆▇▇▇
val_loss,█▇▆▄▅▄▃▃▂▂▄▄▂▂▂▂▂▂▂▂▂▁▂▁▁▁▃▁▂▂▂▂▁▁▁▁▃▂▂▂
accuracy,0.88889
epoch,10001
loss,0.67107
val_accuracy,0.92121
val_loss,0.62999


wandb: Agent Starting Run: aex7j9ov with config:
wandb: 	batch_size: 128
wandb: 	conv_depth: 3
wandb: 	conv_kernel: 32
wandb: 	cube_scale: 1
wandb: 	learning_rate: 0.001
wandb: 	rotate_angle: 1
wandb: 	scale_range: 0.03
wandb: 	shift_mul: 0
wandb: 	shift_sigma: 0.05
wandb: 	spacing: [16, 16, 16]
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /content/drive/MyDrive/Colab Notebooks/1_work_experiments_wandb.ipynb.


100%|██████████| 10/10 [00:00<00:00, 58.14it/s]


[E:0,T:0.0] 

[E:20,T:2.4] loss=1.0047149658203125, accuracy=0.4545454680919647, val_loss=1.0125547647476196, val_accuracy=0.4590908885002136, 

[E:40,T:3.1] loss=0.8727813363075256, accuracy=0.6565656661987305, val_loss=0.8878541588783264, val_accuracy=0.6469696760177612, 

[E:60,T:3.7] loss=0.8221754431724548, accuracy=0.7171717286109924, val_loss=0.9090224504470825, val_accuracy=0.6257575750350952, 

[E:80,T:4.4] loss=0.8350940346717834, accuracy=0.6767676472663879, val_loss=0.8940443396568298, val_accuracy=0.6439393758773804, 

[E:100,T:5.1] loss=0.8777961134910583, accuracy=0.6363636255264282, val_loss=0.9052953124046326, val_accuracy=0.6242424249649048, 

[E:120,T:5.7] loss=0.8500661253929138, accuracy=0.6868686676025391, val_loss=0.876251757144928, val_accuracy=0.653030276298523, 

[E:140,T:12.3] loss=0.8714316487312317, accuracy=0.6464646458625793, val_loss=0.8851845264434814, val_accuracy=0.6424242258071899, 

[E:160,T:12.9] loss=0.8097435235977173, accuracy=0.747474730014801,